# Demo Generation
* model：GLM-4
* psychological counseling report + counseling note to CPsyCounD

In [ ]:
import threading
import json
import os
from tqdm import tqdm
import concurrent.futures
import requests
from dotenv import load_dotenv
from zhipuai import ZhipuAI

In [ ]:
system_message = '''
# Role
你是一位拥有二十年从业经验的心理咨询师，擅长重建心理咨询场景。
## Attention
您负责基于心理咨询报告和咨询笔记，还原来访者和心理咨询师的多轮长对话。

## Skills
### skill 1：真实性表达
- 来访者多情绪化表达，符合真实心理咨询场景
- 心理咨询师采取引导式对话，倾听、理解、支持来访者
- 来访者与心理咨询师避免长篇表述，单轮对话尽可能少于50字

### skill 2：咨询框架
#### 阶段1：接待询问阶段
- 来访者介绍自己的大致情况，来咨询的目的，想要解决的问题
- 心理咨询师获取来访者的基本信息，包括自我介绍，咨询目的以及期望解决的问题
- 参考咨询笔记的"来访者的基本情况"

#### 阶段2：诊断阶段
- 心理咨询师根据来访者的描述，分析并明确其心理问题，探寻问题的源头和严重程度
- 参考咨询笔记的"来访者的心理问题"

#### 阶段3：咨询阶段
- 心理咨询师与来访者确认咨询目标，告知心理咨询技术
- 分步执行具体执行计划，帮助来访者全方位解决问题。
- 实施咨询笔记的"咨询方案"

#### 阶段4：巩固与结束阶段
- 咨询师与来访者对咨询阶段所做的工作进行回顾和总结，让来访者进行自我反思
- 参考咨询笔记的"经验感想与反思"

## Constrains
- 对话应围绕咨询框架的四个阶段重建，以咨询笔记为依据，提供一段5-15轮的多轮长对话
- 对话以"来访者："开始，"心理咨询师："结束。
- 对话符合真实的心理咨询场景，不得提及咨询报告和咨询笔记本身。

请深呼吸并逐步分析心理咨询报告和咨询笔记，还原来访者和心理咨询师的多轮长对话。
'''

In [ ]:
interrupted = threading.Event()
pbar = None

In [ ]:
def get_completion_from_messages(case_0, case_1, i, system_message):

    try:
        if interrupted.is_set():
            raise KeyboardInterrupt
        
        case_str_0 = json.dumps(case_0, ensure_ascii=False)     # json -> str
        case_str_1 = case_1                # str -> str
        
        if not case_str_0 or not case_str_1:
            print(f"\nError at {i}: invalid")
            with open("error_log.txt", "a") as f:
                f.write(f"\nError at case {i}: invalid")
            return
        
        case_str = system_message + '\n' + '"""心理咨询报告:' + case_str_0  + '\n' + case_str_1 + '"""'

        messages = [{"role": "user", "content": case_str}]
        
        load_dotenv()
        client = ZhipuAI(api_key=os.environ["ZhipuAI_API_KEY"])

        response = client.chat.completions.create(
            model="glm-4",
            messages = messages,
            temperature= 0.75,
            top_p= 0.7,
            incremental=True
        )
        
        with open(f"response_case_{i}.txt", 'w') as f:
            ans = response['data']['choices'][0]['content']
            f.writelines(ans)

    except KeyboardInterrupt:
        print(f"\nError at case {i}: Interrupted")
        with open("error_log.txt", "a") as f:
            f.write(f"\nError at case {i}: Interrupted")
        return
    
    except Exception as e:
        print(f"\nError at case {i}: {e}")
        with open("error_log.txt", "a") as f:
            f.write(f"\nError at case {i}: {e}")

    pbar.update()


def load_case(filepath):
    try:
        extension = filepath.split('.')[-1]
        with open(filepath, 'r') as f:
            if extension == 'json':
                case = json.load(f)
            elif extension == 'txt':
                case = f.read()
            else:
                raise ValueError('未知的文件格式')
        return case

    except Exception as e:
        print(f"\nError at load case {filepath}: {e}")
        with open("error_log.txt", "a") as f:
            f.write(f"\nError at load case {filepath}: {e}")
        return None

In [ ]:
if __name__ == "__main__":
    path_0 = '' # report file path
    path_1 = '' # Memo file path
    threads = []
    pbar = tqdm(total=3133, desc="Processing", dynamic_ncols=True, unit=" case", lock_args=(False,))

    try:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            for i in range(0, 3133):
                case_file_path = os.path.join(path_0, f'case_{i}.json')
                Memo_file_path = os.path.join(path_1, f'response_case_{i}.txt')
                case_0 = load_case(case_file_path)
                case_1 = load_case(Memo_file_path)
                executor.submit(get_completion_from_messages, case_0, case_1, i, system_message)

    except KeyboardInterrupt:
        print("Interrupted by user")
        interrupted.set()

    pbar.close()